In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
from sklearn.metrics import r2_score,mean_squared_error
from collections import Counter

import torchvision

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
stop_word=set(stopwords.words('english'))

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
device


In [ ]:
train=pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
train.head()

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
test=pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
test

In [ ]:
text=train['excerpt']

In [ ]:
sns.jointplot(  x=train['target'], 
                y=train['standard_error'], 
                kind='hex',
                height=7,
                edgecolor="#f5f3ea",
                color= "#496525")
plt.suptitle("Standard and target error ", size=15)
plt.subplots_adjust(top=0.95)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
sns.histplot(train['target'])
plt.subplot(2,2,2)
sns.boxplot(train['target'],orient='v')
plt.subplot(2,2,3)
sns.histplot(train['standard_error'])

In [ ]:
import re
def preprocessing(s):
    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespaces with no space
    s = re.sub(r"\s+", '', s)
    # replace digits with no space
    s = re.sub(r"\d", '', s)
    
    return s

In [ ]:
text=[i.lower().split()  for i in text]
text_new=[]
for sen in text:
    lines=[]
    for i in  sen:
        i=re.findall(r"[\w]+|[.,!?;]", preprocessing(i)) #we separate punctuation marks from words
        lines.append(i)
    text_new.append(np.concatenate(np.array(lines,dtype=object)))

In [ ]:
word_list=[]
for sen in text_new:
    for word in sen:
        if word not in stop_word:
            word_list.append(word)
      

In [ ]:
c=Counter(word_list)
corpus=sorted(c,key=c.get,reverse=True)[:1500]
one_hot_dict={ w:i+1 for i,w in enumerate(corpus)} 
train_list=[]
for k,sentence in enumerate(text_new):
    train_list.append( [one_hot_dict[word] for i,word in enumerate (sentence)
                           if word in one_hot_dict.keys()])
pad_len=max([len(i) for i in train_list]) #max pad 

In [ ]:
#padding
pad=pad_len
def padding(train_list,padd):
    feature=np.zeros((len(train_list),padd),dtype=int)
    for i,sentence in enumerate(train_list) :
        feature[i,-len(sentence):]=np.array(sentence)[:pad]
    return feature

text_end=padding(train_list,pad)

In [ ]:
batch_size=32
target=train['target'].values
train_data= TensorDataset(torch.from_numpy(text_end)[:-900], torch.from_numpy(target)[:-900])
val_data= TensorDataset(torch.from_numpy(text_end)[-900:], torch.from_numpy(target)[-900:])

train_loader=DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader=DataLoader(val_data, shuffle=True, batch_size=batch_size)

# obtain one batch of training data
dataiter = iter(train_loader)
text_e, trgt = dataiter.next()

print('Sample input size: ', text_e.size()) # batch_size, seq_length
print('Sample input: \n', text_e)
print('Sample input: \n', trgt)

In [ ]:
class LSTMsentiment(nn.Module):
    def __init__(self,vocab_size,embed_size,hidden_size,pad):
        super(LSTMsentiment,self).__init__()
        
        self.hidden=hidden_size
        self.embed_size=embed_size
        self.num_layers=1
        self.pad=pad
        self.embed=nn.Embedding(vocab_size,embed_size)
        self.lstm=nn.LSTM(input_size= self.embed_size,
                          hidden_size=self.hidden,
                          num_layers=1,
                          
                          batch_first=True)
        
        
        self.fc=nn.Linear(self.hidden,128)
        self.fc2=nn.Linear(128,1)
        #self.fc3=nn.Linear(64,1)


        self.drop=nn.Dropout(0.9)
        self.relu=nn.ReLU()
        
        
        
    def forward(self,x):
    
        batch=x.shape[0]
        x=self.embed(x)
        x,hidden =self.lstm(x)
        x=x.contiguous().view(-1, self.hidden)
        
        x=self.drop(x)
        x=self.relu(x)
        x=self.fc(x)
        x=self.drop(x)
        x=self.relu(x)
        x=self.fc2(x)
        #x=self.relu(x)
        #x=self.fc3(x)
        out=x.view(batch,self.pad,-1)
        
        return out[:,-1]



#    def init_hidden(self,batch):

#        h0=torch.zeros((self.num_layers,batch,self.hidden)).to(device)
#        c0=torch.zeros((self.num_layers,batch,self.hidden)).to(device)

#        return h0,c0



         


In [ ]:
pad

In [ ]:
vocab_size=len(word_list)
embed_size=200
hidden_size=512
lr=0.001
model=LSTMsentiment(vocab_size,embed_size,hidden_size,pad).float().to(device)
criterion=nn.MSELoss(reduction='sum')
optimizer=torch.optim.Adam(model.parameters(),lr=lr)


epoch_pred={}
for epoc in range(50):
    #hidden_unit=model.init_hidden(batch_size)
    train_loss=0
    train_preds=[]
    train_labels=[]
    model.train()
    for i,(data,label) in enumerate(train_loader):
      
        train_data=data.to(device)
        train_label=label.to(device)
        
        
       # h=tuple([each.data for each in hidden_unit])
        train_pred=model(train_data)
        loss=criterion(train_pred.float(),train_label.float().reshape(-1,1))
        train_loss=train_loss+loss.item()


        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_labels.append(train_label.cpu().detach().numpy())
        train_preds.append(train_pred.cpu().detach().numpy())

    #epoch_pred.update({epoc:preds})
    train_l=np.concatenate(train_labels).reshape(-1,1)
    train_p=np.concatenate(train_preds)
    train_r2=r2_score(train_l,train_p)
    
    model.eval()
    val_labels=[]
    val_preds=[]
    
    for i,(val_data,val_label) in enumerate(val_loader):
         
        val_data=val_data.to(device)
        val_label=val_label.to(device)
        
        val_pred=model(val_data)

        val_labels.append(val_label.cpu().detach().numpy())
        val_preds.append(val_pred.cpu().detach().numpy())       
        
    #epoch_pred.update({epoc:preds})
    val_l=np.concatenate(val_labels).reshape(-1,1)
    val_p=np.concatenate(val_preds)
    val_r2=r2_score(val_l,val_p)
    
    print('epoch: {:}, Train_rmse: {:.5f} , Train_r2: {:.3f} --- Val_rmse: {:.5f} '.
          format(epoc,np.sqrt(np.sum((train_p-train_l)**2)/len(train_l)),train_r2,
                      np.sqrt(np.sum((val_p-val_l)**2)/len(val_l))))


In [ ]:
test=pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
test

In [ ]:
test_text=test['excerpt']
test_text

In [ ]:
text_t=[i.lower().split()  for i in test_text]
text_t_new=[]
for sen in text_t:
    lines=[]
    for i in  sen:
        i=re.findall(r"[\w]+|[.,!?;]", preprocessing(i)) #we separate punctuation marks from words
        lines.append(i)
    text_t_new.append(np.concatenate(np.array(lines,dtype=object)))

In [ ]:
#c=Counter(word_list)
#corpus=sorted(c,key=c.get,reverse=True)
#one_hot_dict={ w:i+1 for i,w in enumerate(corpus)} 
test_list=[]
for k,sentence in enumerate(text_t_new):
    test_list.append( [one_hot_dict[word] for i,word in enumerate (sentence)
                           if word in one_hot_dict.keys()])
max([len(i) for i in test_list]) #max pad 



In [ ]:
text_test_end=padding(test_list,pad)
test_end=torch.tensor(text_test_end).to(device)

In [ ]:
test_end.shape

In [ ]:
model.eval()
with torch.no_grad():
     output=model(test_end).to(device)
output

In [ ]:
sample_submission=pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')
sample_submission['target']=output.cpu().detach().numpy()
sample_submission

In [ ]:
sample_submission.index=sample_submission['id']
sample_submission.drop('id',axis=1,inplace=True)

In [ ]:
sample_submission.to_csv('/kaggle/working/submission.csv')

In [ ]:
sample_submission
